In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [27]:
rating_dict = {}

for tr in rating_table:
    header_td = tr.find('td', class_='review-rating-header')
    stars_td = tr.find('td', class_='review-rating-stars')

    if header_td and stars_td:
        header = header_td.get_text(strip=True)  
        fill_stars = stars_td.find_all('span', class_='fill')  
        count_fill_stars = len(fill_stars)  
        rating_dict[header] = count_fill_stars

print(rating_dict)


{'Seat Comfort': 4, 'Cabin Staff Service': 5, 'Food & Beverages': 5, 'Inflight Entertainment': 5, 'Ground Service': 4, 'Wifi & Connectivity': 5, 'Value For Money': 4}


In [57]:
full_review = list()
counts = 0 
for i in range(1, 31):
    ready_response = requests.get(f'https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize=100')
    if ready_response.status_code == 200:
        ready_soup = BeautifulSoup(ready_response.text, 'html.parser')  
    else:
        print(f"Error: Unable to fetch the content:{ready_response.status_code}")

    all_article_in_a_page = ready_soup.find_all('article', {"itemprop": "review"})
    pattern = r'\((.*?)\)'

    for _, article in enumerate(all_article_in_a_page):
        table = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'review-stats'}).find('table').find_all('tr')
        
        review_dict = {}
        for tr in table:
            header_td = tr.find('td', class_='review-rating-header')
            value_td = tr.find('td', class_='review-value')
            stars_td = tr.find('td', {'class': 'review-rating-stars'})
            
            if header_td and value_td:
                header = header_td.get_text(strip=True)
                value = value_td.get_text(strip=True)
                review_dict[header] = value
            elif not value_td:
                header = header_td.get_text(strip=True)
                stars = stars_td.get_text(strip=True)
                fill_stars = stars_td.find_all('span', class_='fill')
                review_dict[header] = len(fill_stars)

        
        is_verified = article.find('div', {'class': 'body'}).find('div', {'class': 'tc_mobile'}).find('div', {'class': 'text_content'}).find('strong')   
        is_verified_ready = is_verified.text if is_verified is not None else None
        country_of_origin = re.findall(pattern, article.find('div', {'class': 'body'}).find('h3').text)[0]
        country_of_origin_ready = country_of_origin.text if country_of_origin is not None else None
        detailed_review = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'}).text.split('|')[1].lstrip() 
        detailed_review_ready = detailed_review.text if detailed_review is not None else None
        
        
        full_review.append({
            'id': counts,
            'review_header': article.find('div', {'class': 'body'}).find('h2', {'class': 'text_header'}).text.strip('"'),
            'date': article.find('div', {'class': 'body'}).find('h3').find('time').text,
            'country_of_origin': country_of_origin,
            'passenger_name': article.find('div', {'class': 'body'}).find('h3').find('span').text.removeprefix('\n').removesuffix('\n').split('\n')[-1],
            'is_verified': is_verified,
            'detailed_review': detailed_review,
            'rating_dict': review_dict,
        })

        # try:
        #     full_review.append({
        #         'id': counts,
        #         'review_header': article.find('div', {'class': 'body'}).find('h2', {'class': 'text_header'}).text.strip('"'),
        #         'date': article.find('div', {'class': 'body'}).find('h3').find('time').text,
        #         # 'country_of_origin': re.findall(pattern, article.find('div', {'class': 'body'}).find('h3').text)[0],
        #         'passenger_name': article.find('div', {'class': 'body'}).find('h3').find('span').text.removeprefix('\n').removesuffix('\n').split('\n')[-1],
        #         'is_verified': article.find('div', {'class': 'body'}).find('div', {'class': 'tc_mobile'}).find('div', {'class': 'text_content'}).find('strong').text,
        #         # 'detailed_review': article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'}).text.split('|')[1].lstrip(),
        #         'rating_dict': review_dict,
        #     })
        # except:
        #     full_review.append(None)
    
        counts+=1
    

AttributeError: 'str' object has no attribute 'text'

In [ ]:
len(full_review)

In [54]:
full_review[10: 20]

[{'id': 10,
  'review_header': 'a very comfortable flight',
  'date': '31st July 2023',
  'passenger_name': 'Rajan Parrikar',
  'is_verified': <strong><a href="https://www.airlinequality.com/verified-reviews/"><em>Trip Verified</em></a></strong>,
  'rating_dict': {'Aircraft': 'Boeing 787-8',
   'Type Of Traveller': 'Couple Leisure',
   'Seat Type': 'Business Class',
   'Route': 'London to Mumbai',
   'Date Flown': 'July 2023',
   'Seat Comfort': 4,
   'Cabin Staff Service': 5,
   'Food & Beverages': 5,
   'Inflight Entertainment': 3,
   'Ground Service': 4,
   'Value For Money': 4,
   'Recommended': 'yes'}},
 {'id': 11,
  'review_header': 'good onboard service',
  'date': '31st July 2023',
  'passenger_name': 'Rajan Parrikar',
  'is_verified': <strong><a href="https://www.airlinequality.com/verified-reviews/"><em>Trip Verified</em></a></strong>,
  'rating_dict': {'Aircraft': 'A320',
   'Type Of Traveller': 'Couple Leisure',
   'Seat Type': 'Business Class',
   'Route': 'Keflavik  to Lo

In [30]:
all_article_in_a_page = soup.find_all('article', {"itemprop": "review"})
article_dict = dict()
pattern = r'\((.*?)\)'
counts = 0 
for i, article in enumerate(all_article_in_a_page):
    table = article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'review-stats'}).find('table').find_all('tr')
    
    review_dict = {}
    for tr in table:
        header_td = tr.find('td', class_='review-rating-header')
        value_td = tr.find('td', class_='review-value')
        stars_td = tr.find('td', {'class': 'review-rating-stars'})
        
        if header_td and value_td:
            header = header_td.get_text(strip=True)
            value = value_td.get_text(strip=True)
            review_dict[header] = value
        elif not value_td:
            header = header_td.get_text(strip=True)
            stars = stars_td.get_text(strip=True)
            fill_stars = stars_td.find_all('span', class_='fill')
            review_dict[header] = len(fill_stars)
    
    article_dict[counts] = {
        'review_header': article.find('div', {'class': 'body'}).find('h2', {'class': 'text_header'}).text.strip('"'),
        'date': article.find('div', {'class': 'body'}).find('h3').find('time').text,
        'country_of_origin': re.findall(pattern, article.find('div', {'class': 'body'}).find('h3').text)[0],
        'passenger_name': article.find('div', {'class': 'body'}).find('h3').find('span').text.removeprefix('\n').removesuffix('\n').split('\n')[-1],
        'is_verified': article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'}).find('strong').text,
        'detailed_review': article.find('div', {'class': 'body'}).find('div').find('div', {'class': 'text_content'}).text.split('|')[1].lstrip(),
        'rating_dict': review_dict,
    }

    counts+=1

In [31]:
from pprint import pprint
pprint(article_dict)

{0: {'country_of_origin': 'United Kingdom',
     'date': '13th August 2023',
     'detailed_review': 'Easy check in a T5. Galleries south and North lounges '
                        'packed, only just able to get a seat. Didn’t stay '
                        'long as B gates lounge much quieter. Flight boarded '
                        'smoothly and on time. A380 upper deck. Old club world '
                        'seating which is beginning to age a little. However, '
                        'like the large toilets on this aircraft type. Cabin '
                        'crew were all fantastic. One family had tech issues '
                        'with their seats and entertainment systems which crew '
                        'worked hard on though sadly without success. Food and '
                        'drink options were all very good. Entertainment was '
                        'good. Overall an enjoyable flight.',
     'id': 0,
     'is_verified': 'Trip Verified',
     'passeng

In [23]:
new_dict = {}

In [22]:
new_dict

{'now': 'happy'}